In [1]:
import os
import numpy as np
import scipy.io as sio
import h5py
import pandas as pd

N_SAMPLE = 269648

In [23]:
print("--- label ---")
LABEL_P = '/Users/zhangxiao/Desktop/多模态数据集/NUS-wide/Groundtruth/AllLabels_81'

# class order determined by `Concepts81.txt`
cls_id = {}
cls_id_r = {}
with open('/Users/zhangxiao/Desktop/多模态数据集/NUS-wide/Groundtruth/Concepts81.txt', 'r') as f:
    for cid, line in enumerate(f):
        cn = line.strip() 
        cls_id[cn] = cid       
        cls_id_r[cid] = line.strip()       
        
# print("\nclass-ID:", cls_id)
id_cls = {cls_id[k]: k for k in cls_id}
# print("\nID-class:", id_cls)

class_files = os.listdir(LABEL_P)
# print("\nlabel file:", len(class_files), class_files)
label_key = lambda x: x.split('.txt')[0].split('Labels_')[-1]

N_CLASS = len(cls_id)
print("\n#classes:", N_CLASS)
class_files = os.listdir(LABEL_P)

# print("\nlabel file:", len(class_files), class_files)
label_key = lambda x: x.split(".txt")[0].split("Labels_")[-1]

labels = np.zeros([N_SAMPLE, N_CLASS], dtype=np.int8)
for cf in class_files:
    c_name = label_key(cf)
    cid = cls_id[c_name]
#     print('->', cid, c_name)
    with open(os.path.join(LABEL_P, cf), 'r') as f:
        for sid, line in enumerate(f):
            if int(line) > 0:
                labels[sid][cid] = 1
# print("labels:", labels.shape, ", cardinality:", labels.sum())
# labels: (269648, 81) , cardinality: 503848
# np.save("labels.npy", labels.astype(np.int8))
labels = labels.astype(np.uint8)
# sio.savemat("labels.mat", {"labels": labels}, do_compression=True)

--- label ---

#classes: 81


In [24]:
# print(cls_id)
# print(cls_id_r)

In [25]:
# labels[595]

In [26]:
single_label = []
single_label_sam = [] 
for i in range(labels.shape[0]):
    if labels[i].sum() == 1:
        single_label.append(i)
        single_label_sam.append(labels[i])

In [27]:
# single_label

In [28]:
# len(single_label)
# print(labels[1052])
# len(single_label_sam)

In [29]:
# single_label_sam[0]

In [30]:
Train_label = []
Train_label_sam = {}
Test_label = []
Test_label_sam = {}

In [31]:
import random
random.seed(8)
Train_label = random.sample(single_label, 24000)
Train_label.sort()
Test_label = random.sample(single_label, 6000)
Test_label.sort()

In [32]:
# print(Train_label)

In [33]:
# print(len(Train_label))
# print(len(Test_label))

In [34]:
# print(Train_label)

In [35]:
# Train_label.index(82173)

In [36]:
# print(Train_label_sam[0])#到这里已经将2.4w个测试样本的label给对立起来，总共81个类

In [37]:
# print(labels[50833])

In [38]:
# len(Train_label_sam)

In [39]:
# for line in Test_label:
#     Test_label_sam[line] = labels[line]

In [40]:
# len(Test_label_sam)

In [41]:
# Train_label_sam = np.array(Train_label_sam)

In [42]:
# Test_label_sam = np.array(Test_label_sam)

In [43]:
# Train_label_sam[0]

In [44]:
# image = pd.read_csv('/Users/zhangxiao/Desktop/多模态数据集/NUS-wide/ImageList/Imagelist.txt', header=None)

In [45]:
# len(image)

In [46]:
# df = pd.read_csv('/Users/zhangxiao/Desktop/多模态数据集/Low_Level_Features/Normalized_CH.dat', sep=' ', header=None)

In [47]:
# df.head()

In [48]:
# print(len(Train_label))

### Tags

In [49]:
tag1k = pd.read_csv('/Users/zhangxiao/Desktop/多模态数据集/NUS-wide/NUS_WID_Tags/未命名文件夹/AllTags1k.txt', sep='\t', header=None)

In [50]:
tag1k.shape

(269648, 1001)

In [51]:
# tag1k.head()

In [52]:
tag1k_list = {}
with open('/Users/zhangxiao/Desktop/多模态数据集/NUS-wide/NUS_WID_Tags/未命名文件夹/TagList1k.txt', 'r') as f:
    for line, tag in enumerate(f):
        tag_id = line
        tag1k_list[tag_id] = tag.strip()

In [53]:
# tag1k_list

In [54]:
# tag1k_list[25]

In [55]:
Train_alltags = {}
with open('/Users/zhangxiao/Desktop/多模态数据集/NUS-wide/NUS_WID_Tags/未命名文件夹/AllTags1k.txt', 'r') as f:
    for line, tags in enumerate(f):
#         print(line)
#         print(type(line))
        if line in Train_label:
#             print(line)
            Train_alltags[line] = np.array(tags.split(), dtype=int)
#             print(alltags)

In [56]:
len(Train_alltags)

24000

In [57]:
Test_alltags = {}
with open('/Users/zhangxiao/Desktop/多模态数据集/NUS-wide/NUS_WID_Tags/未命名文件夹/AllTags1k.txt', 'r') as f:
    for line, tags in enumerate(f):
#         print(line)
#         print(type(line))
        if line in Test_label:
#             print(line)
            Test_alltags[line] = np.array(tags.split(), dtype=int)
#             print(alltags)

In [58]:
Train_tags = {}
Train_tag = []
for line, tags in Train_alltags.items():
#     print(list(np.where(tags==1))[0])
    for index in list(np.where(tags==1))[0]:
        Train_tag.append(tag1k_list[index])
    Train_tags[line] = Train_tag
    Train_tag = []

In [59]:
del_train = [] # 删除一些没有tag的样本
for index, tags in list(Train_tags.items()):
    if len(tags) == 0:
        del Train_tags[index]
        del_train.append(index)

In [60]:
for index in del_train:
    if index in Train_label:
        Train_label.remove(index)

In [61]:
Test_tags = {}
Test_tag = []
for line, tags in Test_alltags.items():
#     print(list(np.where(tags==1))[0])
    for index in list(np.where(tags==1))[0]:
        Test_tag.append(tag1k_list[index])
    Test_tags[line] = Test_tag
    Test_tag = []

In [62]:
del_test = [] # 删除一些没有tag的样本
for index, tags in list(Test_tags.items()):
    if len(tags) == 0:
        del Test_tags[index]
        del_test.append(index)

In [63]:
for index in del_test:
    if index in Test_label:
        Test_label.remove(index)

In [64]:
print(len(Test_label))

5774


In [65]:
Test_label[0]

19

In [66]:
# Test_tags[167242]

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [68]:
Train_tags_feature = []
Test_tags_feature = []
for index,tags in Train_tags.items():
#     print(tags)
    Train_tags_feature.append(' '.join(tags))
#     print(Train_tags_feature)
for index,tags in Test_tags.items():
    Test_tags_feature.append(' '.join(tags))

Train_transfer = TfidfVectorizer()
Train_tags_final = Train_transfer.fit_transform(Train_tags_feature)
# print('Train_tags_feature:\n', Train_tags_final.toarray())
# print('特征名字:\n', transfer.get_feature_names())
Test_transfer = TfidfVectorizer()
Test_tags_final = Test_transfer.fit_transform(Test_tags_feature)

In [69]:
print(Train_tags_final.toarray().shape)
print(len(Train_transfer.get_feature_names()))
print(Train_tags_final.toarray()[0])

(23074, 1000)
1000
[0.         0.         0.         0.65134012 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0. 

In [70]:
print(Test_tags_final.toarray().shape)
print(len(Test_transfer.get_feature_names()))
print(Test_tags_final.toarray()[0])

(5774, 1000)
1000
[0.         0.         0.         0.37099589 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.26625357 0.
 0.         0.         0.         0.  

In [71]:
Test_tags_feature

['black film new moon twilight actor movie',
 'portrait beautiful london star news actor gorgeous',
 'woman india actor movie',
 'sheep actor',
 'woman india smile sweet actor lady mall',
 'people portrait men girls dress group hat boys adults actor males clothing',
 'color film beauty actor',
 'portrait scotland actor',
 'news actor',
 'actor',
 'art old vintage brown men male illustration clothes actor artist pair',
 'people man boy actor friend',
 'actor movie',
 'travel man japan digital festival star japanese parade actor historical',
 'star american actor',
 'school high actor',
 'asia india actor movie youth',
 'green portrait texas dance actor austin',
 'film beauty actor',
 'actor movie',
 'red india photographer model culture female actor traditional',
 'actor',
 'film berlin stars actor',
 'drawing paper actor',
 'portrait vintage actor',
 'people night portrait color man men religion male portraits live actor males faces pope',
 'male march actor',
 'portrait actor',
 'film

### CH 64

In [72]:
CH_P = '/Users/zhangxiao/Desktop/多模态数据集/NUS-wide/Low_Level_Features/Normalized_CH.dat'
ch64 = pd.read_csv(CH_P, sep = ' ', header=None)

In [73]:
# ch64

In [74]:
ch64 = ch64.drop(columns=64) 

In [75]:
# ch64

In [76]:
Train_ch64 = {}
for index, row in ch64.iterrows():
#     print(type(row))
#     print(type(index))
    if index in Train_label:
        line = np.array(row)
        Train_ch64[index] = np.array(row, dtype=float)

In [77]:
# print(Train_ch64[0])

In [78]:
len(Train_ch64)

23074

In [79]:
Test_ch64 = {}
for index, row in ch64.iterrows():
#     print(type(row))
#     print(type(index))
    if index in Test_label:
        line = np.array(row)
        Test_ch64[index] = np.array(row, dtype=float)

###  Tags CH64 Label拼接

In [80]:
# #Tags TF-IDF向量
# print(Train_tags_final.toarray())
# CH64
# print(Train_ch64)
# #Label
# print(Train_label_sam)

In [81]:
Train_CH64 = np.zeros((len(Train_ch64), 64))
i_1 = 0
for value in Train_ch64.values():
    Train_CH64[i_1] = value
    i_1 += 1

Test_CH64 = np.zeros((len(Test_ch64), 64))
i_2 = 0
for value in Test_ch64.values():
    Test_CH64[i_2] = value
    i_2 += 1

In [82]:
# print(Train_CH64[0])

In [83]:
for line in Train_label:
    Train_label_sam[line] = labels[line] 
Train_Labels = np.zeros((len(Train_label_sam),81))
j_1 = 0
for value in Train_label_sam.values():
    Train_Labels[j_1] = value
    j_1+=1

for line in Test_label:
    Test_label_sam[line] = labels[line] 
Test_Labels = np.zeros((len(Test_label_sam),81))
j_2 = 0
for value in Test_label_sam.values():
    Test_Labels[j_2] = value
    j_2+=1

In [101]:
print(Train_Labels[:5])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.

In [85]:
Train_Data = np.hstack((Train_CH64, Train_tags_final.toarray(), Train_Labels))

In [86]:
print(Train_Data[0])
print(Train_Data[0].shape)

[-0.047583  0.03591  -0.021328 ...  0.        0.        0.      ]
(1145,)


In [87]:
Test_Data = np.hstack((Test_CH64, Test_tags_final.toarray(), Test_Labels))

In [88]:
print(Test_Data[0])
print(Test_Data[0].shape)

[-0.065481  0.222105  0.021829 ...  0.        0.        0.      ]
(1145,)


In [94]:
print(np.array(np.where(Train_Labels[0]==1)))

[[42]]


In [97]:
Train_Label = np.zeros(len(Train_Labels))
for i in range(len(Train_Labels)):
    Train_Label[i] = (np.array(np.where(Train_Labels[i]==1)))
    

In [104]:
print(Train_Label.reshape((len(Train_Labels), -1)).astype(int))

[[42]
 [42]
 [42]
 ...
 [ 1]
 [75]
 [ 1]]


In [10]:
NUS = np.array([[1, 2, 3, 4, 5], [3, 4, 5, 2, 1], [8, 5, 6, 7, 2]])

In [11]:
NUS

array([[1, 2, 3, 4, 5],
       [3, 4, 5, 2, 1],
       [8, 5, 6, 7, 2]])

In [14]:
M0 = np.zeros((len(NUS), len(NUS)))
result = []
for i in range(len(NUS)):
    X = NUS[i]
    for j in range(i, len(NUS)):
        Z = NUS[j]
        for l in range(len(NUS[0])):
            if X[l] < Z[l]:
                result.append(X[l])
            else:
                result.append(Z[l])
        M0[i][j] = sum(result)
        M0[j][i] = sum(result)
        result = []

In [15]:
M0

array([[15.,  9., 12.],
       [ 9., 15., 15.],
       [12., 15., 28.]])

In [20]:
M1 = np.zeros((len(NUS), len(NUS)))
for i in range(len(NUS)):
    X = NUS[i, :]
    for j in range(i, len(NUS)):
        Z = NUS[j, :]
        result = np.dot(X, Z)/(np.linalg.norm(X) * np.linalg.norm(Z))
        M1[i][j] = result
        M1[j][i] = result

In [21]:
M1

array([[1.        , 0.70909091, 0.74789456],
       [0.70909091, 1.        , 0.90960149],
       [0.74789456, 0.90960149, 1.        ]])

In [106]:
no = np.random.normal(loc=0.0, scale=1.0, size=(3,2))

In [107]:
no

array([[-0.14414968,  0.30355835],
       [-0.04071123,  0.78350292],
       [-1.76360429, -0.59919128]])

In [2]:
o_1 = np.array(([[0, 1, 0, 0, 0, 0],
                 [1, 0, 0, 0, 0, 0],
                 [0, 0, 0, 1, 0, 0],
                 [0, 0, 0, 0, 1, 0]]))

In [6]:
np.where(o_1[]==1)

(array([0, 1, 2, 3]), array([1, 0, 3, 4]))

In [7]:
o_s = np.zeros(len(o_1))
for i in range(len(o_1)):
    o_s[i] = (np.array(np.where(o_1[i]==1)))

In [8]:
o_s

array([1., 0., 3., 4.])

In [10]:
o_s.reshape((len(o_1), -1))

array([[1.],
       [0.],
       [3.],
       [4.]])